In [56]:
import numpy as np
import pandas as pd

In [60]:
df = pd.read_csv('P:/final_prchs_data.csv')

In [61]:
df.columns

Index(['prchs_id', 'prod_id', 'partition_dt', 'insd_usermbr_no', 'prod_amt',
       'sex_clsf_cd', 'age_cd', 'category_nm', 'dtl_category_nm',
       'prchs_tm_clsf_nm', 'mno_cd', 'cust_payment_amt', 'sett_target_cpn_amt',
       'pay_mtd_cnt'],
      dtype='object')

#### partition_dt를 datetime으로

In [63]:
#yyyy-mm-dd면 format='%Y-%m-%d'

#날짜를 datetime으로 바꾸기
df['partition_dt'] = pd.to_datetime(df['partition_dt'], format='%Y%m%d', errors='raise')

In [64]:
#주말인지 아닌지
df['is_weekend'] = df['partition_dt'].apply(lambda x: 0 if x.weekday() < 5 else 1)
#month 변수추가
df['month'] = df['partition_dt'].dt.month
#week 변수추가
df['week'] = df['partition_dt'].map(lambda x : x.isocalendar()[1])

# #공짜로 샀는지 아닌지
# df['is_free'] = df['cust_payment_amt'].apply(lambda x: 1 if x == 0 else 0)
# #공휴일 
# temp = ["2020-03-01", "2020-04-15", "2020-04-30", "2020-05-05", "2020-06-06"]
# df['holiday'] = df.apply(lambda x: 1 if x['partition_dt'] in temp else 0, axis = 1)

## 여기서부터 고객테이블 

In [65]:
#기본 고객 테이블
def create_customer_table(df):
    '''
    return table features:
    'insd_usermbr_no' : 고객id
    'first_prchs_dt' : 기간내 첫 구매 -
    'last_prchs_dt' : 기간내 마지막 구매 -
    'sex_clsf_cd' : 성별 -
    'age_cd' : 나이 -
    'total_prchs_cnt' : 총 결제횟수 -
    'weekend_prchs_cnt' : 주말결제횟수 -
    'total_payment_amt' : 총 결제금액 -
    'total_cpt_amt' : 총 쿠폰 사용금액 -
    'total_prod_amt' : 총 상품 가격 -
    'pref_category' : 선호 카테고리 -
    'pref_dtl_category' : 선호 세부 카테고리  -
    'pref_tm' : 선호 결제 시간 -
    'mno_cd' : 통신사-
    
    '''
    
    print('그룹바이 하는중')
    user_grpby = df.groupby(['insd_usermbr_no'])
    agg_format = {'prod_id':'count',   #개별상품을 거래로 친다. 거래아이디에 여려개의 개별상품이 있을 수 있기때문
                  'cust_payment_amt':'sum',
                  'sett_target_cpn_amt':'sum',
                  'prod_amt': 'sum',
                  'is_weekend':'sum',
                  'partition_dt': 'min', #날짜는 최소값 먼저
                  'sex_clsf_cd': 'min',
                  'age_cd':'max',
                  'mno_cd':'max',
                  'category_nm': lambda x: x.mode()[0],
                  'dtl_category_nm' :lambda x: x.mode()[0],
                  'prchs_tm_clsf_nm':lambda x:x.mode()[0]
                 }
    
    user_grpby_feature = user_grpby.agg(agg_format).reset_index()
    
    print('customer table 생성중')
    
    customer_df = user_grpby_feature[['insd_usermbr_no',
                                    'prod_id',
                                    'mno_cd',
                                    'cust_payment_amt',
                                    'sett_target_cpn_amt',
                                    'prod_amt',
                                    'is_weekend',
                                    'partition_dt',
                                    'sex_clsf_cd',
                                    'age_cd',
                                    'category_nm',
                                    'dtl_category_nm',
                                    'prchs_tm_clsf_nm'
                                    ]]
    
    #feature 이름 변경
    customer_df.rename(columns = {'partition_dt': 'first_prchs_dt',
                                 'prod_id':'total_prchs_cnt',
                                 'cust_payment_amt':'total_payment_amt',
                                 'sett_target_cpn_amt':'total_cpt_amt',
                                 'prod_amt':'total_prod_amt',
                                 'category_nm':'pref_category',
                                 'dtl_category_nm':'pref_dtl_category',
                                 'prchs_tm_clsf_nm':'pref_tm',
                                 'is_weekend':'weekend_prchs_cnt'}, 
                       inplace = True)
    
    
    #last_prchs_dt 추가
    customer_df = customer_df.merge(user_grpby['partition_dt'].max().reset_index())  # 날짜에 max로 마지막 결제일을 구해 추가
    customer_df.rename(columns = {'partition_dt': 'last_prchs_dt'}, 
                       inplace = True)
    
    
    # col 순서바꾸기
    customer_df = customer_df[['insd_usermbr_no','mno_cd',
                               'first_prchs_dt','last_prchs_dt','sex_clsf_cd', 'age_cd',
                               'total_prchs_cnt','weekend_prchs_cnt',
                               'total_payment_amt','total_cpt_amt','total_prod_amt',
                               'pref_category','pref_dtl_category','pref_tm'
                               ]]
    
    print('완')
    
    return customer_df   

In [66]:
#고객테이블 생성
customer_df= create_customer_table(df)

그룹바이 하는중
customer table 생성중
완


In [68]:
customer_df['insd_usermbr_no'].unique().shape

(85314,)

## 돈관련 feature 추가

In [69]:
#일일최대구매
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','partition_dt'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').max())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'dly_prchs_amt_max'}, 
                       inplace = True)

#일일최소구매
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','partition_dt'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').min())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'dly_prchs_amt_min'}, 
                       inplace = True)

#월별최소구매
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','month'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').min())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'mly_prchs_amt_min'}, 
                       inplace = True)

#월별최대구매
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','month'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').max())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'mly_prchs_amt_max'}, 
                       inplace = True)

#월별구매평균
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','month'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').mean())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'mly_prchs_amt_mean'}, 
                       inplace = True)

#주간구매최대
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','week'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').max())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'wly_prchs_amt_max'}, 
                       inplace = True)

#주간구매최소
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','week'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').min())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'wly_prchs_amt_min'}, 
                       inplace = True)


#주간구매평균
temp = pd.DataFrame(df.groupby(['insd_usermbr_no','week'])['cust_payment_amt'].sum().groupby('insd_usermbr_no').mean())
customer_df = customer_df.merge(temp, on = 'insd_usermbr_no', how = 'left')
customer_df.rename(columns = {'cust_payment_amt': 'wly_prchs_amt_mean'}, 
                       inplace = True)

## LifeTimes 
frequency, recency, T 추가

### endtime 수정에 유의

In [70]:
from lifetimes.utils import summary_data_from_transaction_data
summary = summary_data_from_transaction_data(df, 'insd_usermbr_no', 'partition_dt', observation_period_end='2020-07-31')
summary.reset_index()
customer_df = customer_df.merge(summary.reset_index())

## 고객별 공휴일 구매횟수 ---없어서 수행 X

In [71]:
# #공휴일 구매횟수
# temp1 =pd.DataFrame(df.groupby('insd_usermbr_no')['holiday'].sum())

In [72]:
# customer_df = customer_df.merge(temp1.reset_index())

## 정액제 이용고객 이용 횟수 

In [73]:
# 정액제 구매 횟수
data3 = df.loc[(df['category_nm']=="정액제")]
data4 = data3.groupby(['insd_usermbr_no','partition_dt'])['cust_payment_amt'].count().groupby('insd_usermbr_no').sum()
data5 = data4.to_frame().reset_index()
data55 = data5.rename(columns= {'cust_payment_amt': 'total_buys'})
customer_df =pd.DataFrame.merge(customer_df, data55, on = 'insd_usermbr_no', how = 'left')

In [74]:
#정액제 이력이 없는 경우 0으로 채움
customer_df['total_buys'].fillna(int(0), inplace = True)

## 스텝 최소 최대 평균

In [75]:
#각 고객별 이용날짜 리스트구하기
dt_step_temp = pd.DataFrame(df.groupby(['insd_usermbr_no'])['partition_dt'].apply(lambda x:x.unique()))

In [76]:
def cal_min_max_mean_step(timelist):
    timelist.sort()    #이용날짜 리스트를 정렬
    length = len(timelist)
    steplist = []
    
    if length == 1:    # 하루만 이용한 경우
        return 0 , 0 , 0
    elif length == 2:  # 이틀만 이용한 경우
        delta = timelist[1] - timelist[0]
        days = delta.astype('timedelta64[D]')
        step = int(days/np.timedelta64(1, 'D'))
        return step, step, step
    else:              # 사흘이상 이용한 경우
        for i in range(length-1,0,-1): 
            delta = timelist[i] - timelist[i-1]
            days = delta.astype('timedelta64[D]')
            steplist.append(days/np.timedelta64(1, 'D'))
            
        return min(steplist) , max(steplist), round(sum(steplist)/length,1)
    

In [77]:
dt_step_temp['dt_step_min'] , dt_step_temp['dt_step_max'], dt_step_temp['dt_step_mean'] = zip(*dt_step_temp['partition_dt'].apply(lambda x: cal_min_max_mean_step(x)))
del dt_step_temp['partition_dt']

In [78]:
#step결과 병합
customer_df = customer_df.merge(dt_step_temp, on = 'insd_usermbr_no', how = 'left')

In [79]:
#테이블 확인
customer_df

,insd_usermbr_no,mno_cd,first_prchs_dt,last_prchs_dt,sex_clsf_cd,age_cd,total_prchs_cnt,weekend_prchs_cnt,total_payment_amt,total_cpt_amt,...,wly_prchs_amt_max,wly_prchs_amt_min,wly_prchs_amt_mean,frequency,recency,T,total_buys,dt_step_min,dt_step_max,dt_step_mean
0,IF142158896620090701190525,US001201,2020-07-06,2020-07-06,M,046,1,0,8860,1500,...,8860,8860,8860.000000,0.0,0.0,25.0,0.0,0.0,0.0,0.0
1,IF1423000039820090904101650,US001201,2020-07-07,2020-07-31,M,045,5,2,56480,7400,...,27630,8000,18826.666667,2.0,24.0,24.0,0.0,12.0,12.0,8.0
2,IF1423000044820090904173540,US001201,2020-07-02,2020-07-05,M,052,12,5,34200,1800,...,34200,34200,34200.000000,1.0,3.0,29.0,0.0,3.0,3.0,3.0
3,IF1423000080020090908061357,US001201,2020-07-11,2020-07-30,M,038,22,15,46600,7500,...,27400,9300,15533.333333,5.0,19.0,20.0,0.0,1.0,10.0,3.2
4,IF1423000088820090908092140,US001201,2020-07-07,2020-07-07,F,042,1,0,9310,490,...,9310,9310,9310.000000,0.0,0.0,24.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85309,US2020073119334183757417968,US001201,2020-07-31,2020-07-31,F,032,10,0,2910,3090,...,2910,2910,2910.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85310,US2020073120311703057418673,US001206,2020-07-31,2020-07-31,Z,ZZZ,1,0,500,400,...,500,500,500.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85311,US2020073121374433957419409,US001203,2020-07-31,2020-07-31,F,044,49,0,4700,200,...,4700,4700,4700.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85312,US2020073121424175457419483,US001201,2020-07-31,2020-07-31,Z,ZZZ,1,0,0,100,...,0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
#테이블 저장
customer_df.to_csv('P:/파일이름.csv', index = False)